# JEDI - Machine Leaning Workshop

## Aprendizado Supervisionado: Classificação

### Sobre as bibliotecas e os dados

In [ ]:
# Por convenção, as bibliotecas utilizadas devem ser importadas no início do notebook/programa
# Aqui importamos apenas algumas delas, as de uso geral.
# De modo a contextualizar com nossas tarefas, outras seram carregadas ao longo do programa 

# https://numpy.org/doc/stable/user/whatisnumpy.html
# https://pandas.pydata.org/docs/user_guide/index.html#user-guide
# https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.html

import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

Para a implementação do modelo de classificação utilizaremos os dados do da base [*breast cancer*](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)) disponibilizada através da bilioteca de Machine Learning [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html).

A tabela contém métricas extraídas a partir do processamento de imagens do tecido afetado pelo tumor e o resectivo parecer da análise médica: *malignno* ou *benígno*

In [ ]:
# Do módulo de datasets importa a função que carrega a base de dados breast cancer
from sklearn.datasets import load_breast_cancer

# Carrega a base de dados
breast_cancer = load_breast_cancer()

In [ ]:
# TAREFA
# Transforme o dicionário com os dados de cancêr de mama em um DataFrame
df = None
df['target'] = None
nome_dos_alvos = None

In [ ]:
# Quais são os tipos das colunas?

In [ ]:
# Há algum campo vazio no dataset?

In [ ]:
# Quantos dados temos para cada classe?

In [ ]:
# Qual a proporção entre classes?

### Sobre a preparação dos dados

O primeiro passo é separar o que são os atributos e o que é o alvo do modelo.

In [ ]:
df.columns

In [ ]:
X = None # Todas as colunas do dataframe com exceção do target
y = None # Apenas a coluna target

Em seguida temos de separar o que é o conjunto teste e o que é o conjunto treino 

In [ ]:
# Do módulo de model_selection da scikit-learn importa o método train_test_split
# Dados os atributos (X) e o alvo (y), o método train_test_split os separa em conjuntos treino e teste
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.25)

### Sobre o modelo de classificação

Para efetuarmos a tarefa de medição utilizaremos o modelo de [Árvore de Decisão](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) do Scikit-Learn.

In [ ]:
# Do módulo tree importa o modelo DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=4, min_samples_leaf=8, random_state=0)

# Treina o modelo

In [ ]:
# Importa a função plot_decision_tree do script py_utilities
from py_utilities import plot_decision_tree

plot_decision_tree(clf, X.columns, nome_dos_alvos)

### Sobre a avaliação do modelo

A forma mais imediata de avaliação se dá através do método [*score*](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier.score) do classificador, que calculará a acurácia média dados os inputs X e y. 

In [ ]:
print(f'Resultados do modelo: {clf.__str__()}\n')
print(f'\tPerformance nos dados de treino: {clf.score(X_train, y_train):.2f}')
print(f'\tPerformance nos dados de teste : {clf.score(X_test, y_test):.2f}')

Outra possibilidade é utilizarmos o [classafication_report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html), que nos dará uma visão geral da performance do modelo segmentado pelas classes alvo (malígno ou benígno).

In [ ]:
# Do módulo métricas importa a função classification_report 
from sklearn.metrics import classification_report

# Queremos comparar o valor predito pelo modelo com o valor verdadeiro
y_verdadeiro = None
y_predito = None

print(f'Relatório do modelo de classificação {clf.__str__()}\n')
print(classification_report(y_verdadeiro, y_predito, target_names=nome_dos_alvos))

E por fim, uma outra forma de verificarmos o desempenho do nosso classificador é a [matriz de confusão](https://pt.wikipedia.org/wiki/Matriz_de_confus%C3%A3o).

In [ ]:
# Do módulo metric importa a função confusion_matrix
from sklearn.metrics import confusion_matrix

cf_matrix = confusion_matrix(y_verdadeiro, y_predito)

# Plot da matriz de confusão
# Cógido adaptado de https://medium.com/@dtuk81/confusion-matrix-visualization-fc31e3f30fea

group_names = ['Verdadeiro\nNegativo\n', 'Falso\nPositivo\n', 'Falso\nNegativo\n', 'Verdadeiro\nPositivo\n']
group_counts = ['{0:0.0f}'.format(value) for value in
                cf_matrix.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)

sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues');

### Sobre a complexidade do modelo

In [ ]:
# Do módulo metrics importa a métrica f1_score
from sklearn.metrics import f1_score

In [ ]:
max_depth_list = [2, 3, 4, 5, 6, 7, 8, 9]

train_results = []
test_results = []

for depth in max_depth_list:
    clf = DecisionTreeClassifier(max_depth=depth, min_samples_leaf=1, random_state=0)
    clf.fit(X_train, y_train)
    
    score_treino = f1_score(y_train, clf.predict(X_train))
    score_teste = f1_score(y_test, clf.predict(X_test))
    
    train_results.append(score_treino)
    test_results.append(score_teste)
    

plt.figure(figsize=(10, 5))

plt.plot(max_depth_list, test_results, '-o', label='Teste')
plt.plot(max_depth_list, train_results, '-o', label='Treino')

plt.xlabel('Profundidade da Árvore')
plt.ylabel('F1 Score do Modelo')

plt.legend()
plt.show()